In [32]:
import pandas as pd

df = pd.read_csv('/Users/loic.doerr/dev/ny-emergency-ai/data/population/New_York_City_Population_by_Borough__1950_-_2040_20240605.csv')

df.head()

,Age Group,Borough,1950,1950 - Boro share of NYC total,1960,1960 - Boro share of NYC total,1970,1970 - Boro share of NYC total,1980,1980 - Boro share of NYC total,...,2000,2000 - Boro share of NYC total,2010,2010 - Boro share of NYC total,2020,2020 - Boro share of NYC total,2030,2030 - Boro share of NYC total,2040,2040 - Boro share of NYC total
0,Total Population,NYC Total,7891957,100.00,7781984,100.00,7894862,100.00,7071639,100.00,...,8008278,100.00,8242624,100.00,8550971,100.00,8821027,100.00,9025145,100.00
1,Total Population,Bronx,1451277,18.39,1424815,18.31,1471701,18.64,1168972,16.53,...,1332650,16.64,1385108,16.80,1446788,16.92,1518998,17.22,1579245,17.50
2,Total Population,Brooklyn,2738175,34.70,2627319,33.76,2602012,32.96,2230936,31.55,...,2465326,30.78,2552911,30.97,2648452,30.97,2754009,31.22,2840525,31.47
3,Total Population,Manhattan,1960101,24.84,1698281,21.82,1539233,19.50,1428285,20.20,...,1537195,19.20,1585873,19.24,1638281,19.16,1676720,19.01,1691617,18.74
4,Total Population,Queens,1550849,19.65,1809578,23.25,1986473,25.16,1891325,26.75,...,2229379,27.84,2250002,27.30,2330295,27.25,2373551,26.91,2412649,26.73


In [33]:
df = df[['Borough', '2020']]
df = df[df['Borough'] != 'NYC Total']
df = df.rename(columns={'2020': 'Population'})

df.head()

,Borough,Population
1,Bronx,1446788
2,Brooklyn,2648452
3,Manhattan,1638281
4,Queens,2330295
5,Staten Island,487155


In [34]:
import json

with open('./data/geo-data/boroughs.geojson') as rawData:
  boroughs_boundaries_geo = json.load(rawData)

boroughs_df = pd.DataFrame([feature['properties'] for feature in boroughs_boundaries_geo['features']])
boroughs_df = boroughs_df[['boro_name', 'shape_area']]
boroughs_df.columns = ['Borough', 'Area']

df = df.merge(boroughs_df, on='Borough', how='left')

df['Area'] = pd.to_numeric(df['Area'], errors='coerce')

df.head()

,Borough,Population,Area
0,Bronx,1446788,1.187175e+09
1,Brooklyn,2648452,1.934138e+09
2,Manhattan,1638281,6.366456e+08
3,Queens,2330295,3.041419e+09
4,Staten Island,487155,1.623621e+09


In [35]:
df['Population Density'] = (df['Population'] / df['Area']) * 100000

df.head()

,Borough,Population,Area,Population Density
0,Bronx,1446788,1.187175e+09,121.868156
1,Brooklyn,2648452,1.934138e+09,136.931889
2,Manhattan,1638281,6.366456e+08,257.330149
3,Queens,2330295,3.041419e+09,76.618689
4,Staten Island,487155,1.623621e+09,30.004236


In [36]:
fire_house_locations = pd.read_csv('/Users/loic.doerr/dev/ny-emergency-ai/data/locations/FDNY_Firehouse_Listing_20240603.csv')

fire_house_locations.head()

,FacilityName,FacilityAddress,Borough,Postcode,Latitude,Longitude,Community Board,Community Council,Census Tract,BIN,BBL,NTA
0,Engine 4/Ladder 15,42 South Street,Manhattan,10005,40.703694,-74.007717,101,1,7,1000867,1000350001,Battery Park City-Lower Manhattan
1,Engine 6,49 Beekman Street,Manhattan,10038,40.709971,-74.005395,101,1,1501,1001287,1000930030,Battery Park City-Lower Manhattan
2,Manhattan Borough Command/Battalion 1/Engine 7...,100 Duane Street,Manhattan,10007,40.715339,-74.006300,101,1,33,1001647,1001500025,SoHo-TriBeCa-Civic Center-Little Italy
3,Ladder 8,14 N. Moore Street,Manhattan,10013,40.719574,-74.006620,101,1,33,1002150,1001890035,SoHo-TriBeCa-Civic Center-Little Italy
4,Engine 9/Ladder 6,75 Canal Street,Manhattan,10002,40.715408,-73.992834,103,1,16,1003898,1003000030,Chinatown


In [37]:
firehouse_counts = fire_house_locations['Borough'].value_counts()
firehouse_counts.columns = ['Borough', 'Firehouse Quantity']

df = df.merge(firehouse_counts, on='Borough', how='left')

df = df.rename(columns={'count': 'Firehouse Quantity'})

df.head()

,Borough,Population,Area,Population Density,Firehouse Quantity
0,Bronx,1446788,1.187175e+09,121.868156,34
1,Brooklyn,2648452,1.934138e+09,136.931889,66
2,Manhattan,1638281,6.366456e+08,257.330149,48
3,Queens,2330295,3.041419e+09,76.618689,51
4,Staten Island,487155,1.623621e+09,30.004236,20


In [38]:
df['Firehouse Density (Population)'] = (df['Firehouse Quantity'] / df['Population']) * 100000

df.head()

,Borough,Population,Area,Population Density,Firehouse Quantity,Firehouse Density (Population)
0,Bronx,1446788,1.187175e+09,121.868156,34,2.350033
1,Brooklyn,2648452,1.934138e+09,136.931889,66,2.492022
2,Manhattan,1638281,6.366456e+08,257.330149,48,2.929900
3,Queens,2330295,3.041419e+09,76.618689,51,2.188564
4,Staten Island,487155,1.623621e+09,30.004236,20,4.105470


In [39]:
df['Firehouse Density (Area)'] = (df['Firehouse Quantity'] / df['Area']) * 100000

df.head()

,Borough,Population,Area,Population Density,Firehouse Quantity,Firehouse Density (Population),Firehouse Density (Area)
0,Bronx,1446788,1.187175e+09,121.868156,34,2.350033,0.002864
1,Brooklyn,2648452,1.934138e+09,136.931889,66,2.492022,0.003412
2,Manhattan,1638281,6.366456e+08,257.330149,48,2.929900,0.007540
3,Queens,2330295,3.041419e+09,76.618689,51,2.188564,0.001677
4,Staten Island,487155,1.623621e+09,30.004236,20,4.105470,0.001232


In [40]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
df[['Population Density Transformed', 'Firehouse Density (Population) Transformed', 'Firehouse Density (Area) Transformed']] = scaler.fit_transform(df[['Population Density', 'Firehouse Density (Population)', 'Firehouse Density (Area)']])

df['Score'] = df[['Population Density Transformed', 'Firehouse Density (Population) Transformed', 'Firehouse Density (Area) Transformed']].sum(axis=1)

df.head()

,Borough,Population,Area,Population Density,Firehouse Quantity,Firehouse Density (Population),Firehouse Density (Area),Population Density Transformed,Firehouse Density (Population) Transformed,Firehouse Density (Area) Transformed,Score
0,Bronx,1446788,1.187175e+09,121.868156,34,2.350033,0.002864,0.404107,0.084234,0.258752,0.747093
1,Brooklyn,2648452,1.934138e+09,136.931889,66,2.492022,0.003412,0.470372,0.158306,0.345698,0.974375
2,Manhattan,1638281,6.366456e+08,257.330149,48,2.929900,0.007540,1.000000,0.386736,1.000000,2.386736
3,Queens,2330295,3.041419e+09,76.618689,51,2.188564,0.001677,0.205056,0.000000,0.070554,0.275610
4,Staten Island,487155,1.623621e+09,30.004236,20,4.105470,0.001232,0.000000,1.000000,0.000000,1.000000


In [41]:
df.to_csv('./borough_data.csv', index=False)